In [ ]:

!pip install --upgrade urllib3
!pip install flask scikit-learn PyMuPDF pyngrok sentence-transformers

import os
import fitz  # PyMuPDF
from flask import Flask, request, jsonify, render_template_string
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import threading
from pyngrok import ngrok
import socket
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize


def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        pdf_document = fitz.open(pdf_path)
        for page_num in range(len(pdf_document)):
            page = pdf_document.load_page(page_num)
            text += page.get_text()
        return text
    except Exception as e:
        print(f"Error reading PDF: {e}")
        return None

pdf_path = '/content/PYTORCH DOCUMENTATION1.pdf'  # Ensure the PDF is in the correct path
pytorch_text = extract_text_from_pdf(pdf_path)

if pytorch_text is None:
    raise ValueError("Failed to extract text from the PDF document.")


sentences = sent_tokenize(pytorch_text)


model = SentenceTransformer('all-MiniLM-L6-v2')


sentence_embeddings = model.encode(sentences)

def get_response(query):
    try:
        query_embedding = model.encode([query])
        similarities = cosine_similarity(query_embedding, sentence_embeddings).flatten()
        response_index = similarities.argsort()[-10:][::-1]  # Get top 10 most similar sentences
        relevant_sentences = list(dict.fromkeys([sentences[i] for i in response_index]))  # Remove duplicates
        return " ".join(relevant_sentences[:5])  # Return top 5 unique sentences
    except Exception as e:
        print(f"Error in generating response: {e}")
        return "Sorry, I encountered an error while processing your request."

app = Flask(__name__)

@app.route('/')
def home():
    return render_template_string('''
        <!doctype html>
        <html lang="en">
          <head>
            <meta charset="utf-8">
            <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">
            <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.3.1/css/bootstrap.min.css">
            <title>PyTorch Documentation Chatbot</title>
            <style>
              body {
                background-color: #f8f9fa;
                padding-top: 50px;
              }
              .container {
                max-width: 600px;
              }
              .card {
                margin-top: 20px;
              }
              .card-header {
                background-color: #007bff;
                color: white;
              }
            </style>
          </head>
          <body>
            <div class="container">
              <div class="card">
                <div class="card-header">
                  <h3 class="card-title">PyTorch Documentation Chatbot</h3>
                </div>
                <div class="card-body">
                  <form action="/ask" method="post">
                    <div class="form-group">
                      <label for="message">Ask a Question:</label>
                      <input type="text" class="form-control" id="message" name="message" placeholder="Enter your question here" required>
                    </div>
                    <button type="submit" class="btn btn-primary btn-block">Ask</button>
                  </form>
                </div>
              </div>
            </div>
            <script src="https://code.jquery.com/jquery-3.3.1.slim.min.js"></script>
            <script src="https://cdnjs.cloudflare.com/ajax/libs/popper.js/1.14.7/umd/popper.min.js"></script>
            <script src="https://stackpath.bootstrapcdn.com/bootstrap/4.3.1/js/bootstrap.min.js"></script>
          </body>
        </html>
    ''')

@app.route('/ask', methods=['POST'])
def ask():
    try:
        user_input = request.form.get('message')
        if not user_input:
            return "Please provide a valid query."
        response = get_response(user_input)
        return render_template_string('''
            <!doctype html>
            <html lang="en">
              <head>
                <meta charset="utf-8">
                <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">
                <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.3.1/css/bootstrap.min.css">
                <title>PyTorch Documentation Chatbot</title>
                <style>
                  body {
                    background-color: #f8f9fa;
                    padding-top: 50px;
                  }
                  .container {
                    max-width: 600px;
                  }
                  .card {
                    margin-top: 20px;
                  }
                  .card-header {
                    background-color: #007bff;
                    color: white;
                  }
                </style>
              </head>
              <body>
                <div class="container">
                  <div class="card">
                    <div class="card-header">
                      <h3 class="card-title">PyTorch Documentation Chatbot</h3>
                    </div>
                    <div class="card-body">
                      <h4>Response:</h4>
                      <p>{{ response }}</p>
                      <a href="/" class="btn btn-primary btn-block">Ask another question</a>
                    </div>
                  </div>
                </div>
                <script src="https://code.jquery.com/jquery-3.3.1.slim.min.js"></script>
                <script src="https://cdnjs.cloudflare.com/ajax/libs/popper.js/1.14.7/umd/popper.min.js"></script>
                <script src="https://stackpath.bootstrapcdn.com/bootstrap/4.3.1/js/bootstrap.min.js"></script>
              </body>
            </html>
        ''', response=response)
    except Exception as e:
        print(f"Error in /ask route: {e}")
        return f"Internal Server Error: {e}"

@app.route('/chat', methods=['POST'])
def chat():
    try:
        user_input = request.json.get('message')
        if not user_input:
            return jsonify({"response": "Please provide a valid query."})
        response = get_response(user_input)
        return jsonify({"response": response})
    except Exception as e:
        print(f"Error in /chat route: {e}")
        return jsonify({"response": f"Internal Server Error: {e}"})

def find_free_port():
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.bind(('', 0))
    s.listen(1)
    port = s.getsockname()[1]
    s.close()
    return port

port = find_free_port()

def run():
    app.run(host='0.0.0.0', port=port)


thread = threading.Thread(target=run)
thread.start()


!ngrok authtoken 2jC1mWpGT7Q8KEUpqE6f099Hy1g_5yCJoAj1uPvfqNU89tow8


public_url = ngrok.connect(port)
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.4/121.4 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.7
    Uninstalling urllib3-2.0.7:
      Successfully uninstalled urllib3-2.0.7
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 69.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:45301
 * Running on http://172.28.0.12:45301
INFO:werkzeug:Press CTRL+C to quit


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
 * ngrok tunnel "NgrokTunnel: "https://91b7-35-221-255-197.ngrok-free.app" -> "http://localhost:45301"" -> "http://127.0.0.1:45301"
